In [6]:
# IMPORT THE NECESSARY LIBRARIES
import pandas as pd
import re # This module provides functions for working with regular expressions allowing search, match and manipulate strings based on patterns
import os # This module allows interaction with the operating system, such as working with files, directories, environment variables, and system commands.
import sys # This module provides access to system-specific parameters, including command-line arguments (sys.argv), standard input/output, and exiting the program (sys.exit()).
import matplotlib.pyplot as plt
import scipy.optimize as opt
from matplotlib.ticker import MaxNLocator
from statsmodels.graphics.tsaplots import plot_acf # This function is used for plotting the Autocorrelation Function (ACF), which is helpful in time series analysis to measure how observations are correlated with their past values.
from mice_inspection_utils import * # Import all functions 
                                    # NOTICE: this is different from import mice_inspection_utils! In that case you'd have to use the module name as a prefic to access functions, classes or variables. E.g. mice_inspection_utils.some_function()
from scipy.stats import linregress
from glob import glob # Function that will later be used to find all files that match a pattern

# This part avoids possible conflict due to the structure of mice_inspection_utils
if not os.path.exists('Data/by_mouse'): 
    os.makedirs('Data/by_mouse')

# GET THE NECESSARY DATA
imput_subj_paths = [f"Data/raw/total.full.rn.RA.total_OTU_table.mouse{i+1}.ovr0.0p.csv" for i in range(0, 8)]
output_subj_paths = [f"Data/by_mouse/mouse_{i+1}.csv" for i in range(0, 8)]
metadata_path = "Data/raw/OTU_table.csv"
mi = Mice_Inspection(ip = imput_subj_paths, op = output_subj_paths, mdp = metadata_path)

# Population dynamics of bacteria in the gut microbiome: a data analysis

## Short description

The advancement in DNA sequencing tecniques have enabled access to fine-detailed information about the composition of gut microbiota. There is a widespread interest in the scientific comunity around the goal of modeling and predicting the dynamics of these ecological communities, with the aim of producing innovative terapeutic interventions.  However, althogh a wide variety of methods have been published in recent years, many of these are unprincipled and lack reliable validation methods.  Some studies suggest that gut microbiomes cluster into distinct groups (enterotypes), while others argue that these patterns may arise as artifacts of statistical analysis rather than biological reality. 


In our project, following the article *"A macroecological description of alternative stable states reproduces intra- and inter-host variability of gut microbiome", Silvia Zaoli and Jacopo Grilli*, we analyze experimental data of mice gut microbiota assuming a simple stochastic logistic model. This model, specifically, assumes that each bacterial species evolves independently from the others, thus it can be used as a simple, essential null-model to investigate if inter-species interactions can be at all inferred from the data.

The logistic model predicts that, after a transient, the species abundances will eventually settle down to a steady state, parametrized by two parametes: the carrying capacity, accounting for the global effect of the environment, and a parameter $\sigma$ quantifying the stochastic noise. We will follow the methods described in the article, which use a properly defined dissimilarity measure to assess stationarity, designed to overcome the scarsity of data samples.

This main task will require preliminary data processing and basic statistic analysis. 

!!!!   WE HAVEN'T DONE THIS YET !!!!
Optionally, in the case where the null model doesn't suffice to explain the data, we will try to infer the interaction matrix of species-species interaction assuming a simple stochastic Lockta Volterra model, and using linear regression with k-fold validation.

## 1. Preprocessing and sample composition analysis

**Data preprocessing**: 

group OTUs corresponding to same species, fix naming issues, double measures etc, retrieve total number of OTUs, total number of species, set up pandas DataFrames and OTU-species dictionary.

**Analysis of the cross-subject sample composition variability at different taxonomic-unit levels:** 

Biological organisms are classified in groups on the basis of similarity. Biologists refer to these groups as "taxonomic units" or taxa. Different taxa are also grouped together in a higher-level group ("taxon") based on commonly shared features, resulting in a nested hierarcical classification. The basic scheme of modern classification makes use of $8$ levels of classification, which are, from top to bottom: $\textit{Domain, Kindom, Phylum, Class, Order, Family, Genus, Species}$. 

The task is to assess the cross-subject variability of the sample composition at the various levels of the taxonomic classification. The output will be given in the form of stacked-bar plots.

**Rank Abundance Distribution (RAD):**
    
The frequency of each species is computed (= #counts_species/ #counts sample) . Species are ranked with an integer index from the most frequent (i= 1) to the least frequent (i= N). The curve displaying the frequency versus the rank (so-called rank-abundance distribution, or RAD) is experimentally known to assume a lognormal or power-law -like shape, with the first few species occupying the majority of the sample, and a long tail of rare species contributing to the remaining part.

In [1]:
## THE ABOVE PART SHOULD BE REVISITED DEPENDING ON WHAT MIRIAM ACTUALLY WANTS TO TALK ABOUT
## OTHERWISE WE STILL NEED TO MODIFY THE ABOVE TEXT BUT WE CAN SIMPLY PRESENT THE DATA ANALISYS AND FUNCTIONS
## THAT ARE USED FROM MICE_INSPECT LATER

## 2. Theoretical models



#### Deterministic Logistic Model

The deterministic logistic model represents the growth of a population, or in this case the abundance $\lambda$ of a microbial species, constrained by the enviroment. 
The equation for DLM is: $$\dot{\lambda} = \frac{1}{\tau}\lambda \left(1-\frac{\lambda}{K}\right)  $$

This model has two parameters:
* K: the carrying capacity, it is the maximum population size that an enviroment can sustain, given the available resources.
* $\tau$: ralxation time, it is related to the growth rate of the population and determines the time scale of relaxation to stationarity.

1. When $\lambda$ is small, growth is nearly exponential.
2. When $\lambda$ gets close to K, growth slows.
3. When $\lambda=K$, growth stops as $\dot{\lambda}=0$ 



#### Stochastic logistic model

The stochastic version of the model introduces some randomness due to enviromental fluctuatons by adding a noise term, which better reflects real-world variability:
$$\dot{\lambda} = \frac{1}{\tau}\lambda \left(1-\frac{\lambda}{K}\right)  + \lambda \sqrt{\frac{\sigma}{\tau}}\xi(t) $$
where $\xi(t)$ is the Gaussian white noise.

This model has three parameters: 
* K, $\tau$ previously described.
* $\sigma$: it measures the intensity of the enviromental noise. 

The SLM does not include interaction among species and therefore cannot reproduce patterns of interspecies correlation. However, it correctly reproduces several patterns of the dynamics of a single species, including fluctuations and possible extinction events in small populations.

#### Lotka-Volterra model --> !!! TO TAKE OFF SINCE NOT DONE

The Lotka-volterra model extends population dynamics by incorporating interactions between species, such as competition or mutualism. 


## 3. Basic time-series analysis

Say $\vec{X}(t) =\left[x_1(t), x_2(t), \cdots x_N(t)\right]$ is the abudance of the first N most abudant species at time t. We model $\vec{X}(t)$ as a stochastic process, for which
each subject $\vec{X}(t)$ represents a different sample path.

* Visualization:

    Plot time-series for each mouse and for each species to visualize the evolution in time

The basic analysis to perform is:

* Compute the time dependent mean and variance $\mathbb{E}[\vec{X}(t)],\, \mathbb{E}[\vec{X}^2(t)]$. Plot as a summary the time evolution for the mean +- std of each species
* Correlations: filter the most abundant species upon a certain threshold and compute their covariance matrix 
* Autocorrelation: compute the autocorrelation function $\mathbb{E}[x(t) \cdot x(s)]$

In [ ]:
## ALSO THIS IS MIRIAM'S FIELD ##

In [4]:
## JUST AS A COPY OF WHAT THERE IS IN LOGISTICMODEL.IPYNB

### Data Structure

In [7]:
# By calling mice_df we have the dataframes for each mouse
df = mi.mice_df[0]
df.head() # -> As we can see we have the bacterial species, the median, the mean counts, and the counts per day (at day 1, 8, 9 ...)

,Unnamed: 0,species,median_counts,mean_counts,1,8,9,10,15,17,...,1001,1004,1010,1016,1022,1028,1029,1035,1039,1044
0,0,Prevotella sp. Smarlab 121567,412.0,415.156682,0,0,0,2,10,18,...,220,248,252,286,346,293,277,190,222,364
1,1,Parabacteroides distasonis,205.5,206.709677,0,0,0,2,41,145,...,136,157,206,133,177,105,163,147,208,222
2,3,Barnesiella intestinihominis,142.0,145.419355,0,0,0,1,19,58,...,158,145,184,131,145,154,119,130,71,161
3,4,Barnesiella viscericola,125.5,137.092166,0,1,0,1,112,195,...,66,85,134,96,126,121,92,71,72,147
4,2,Lactobacillus taiwanensis,123.5,174.769585,0,422,309,0,48,15,...,36,162,282,174,100,306,517,193,513,175


In [8]:
# By calling get_species_df
df = mi.get_species_df(species = 'Prevotella sp. Smarlab 121567')
df.head(10) # -> As we can see we have the count per day of the selected species in each mouse

,day,mouse_1,mouse_2,mouse_3,mouse_4,mouse_5,mouse_6,mouse_7,mouse_8,mean,std
0,0,0,1,0,0,0,0,0,2,0.375,0.744024
1,1,0,0,0,0,0,0,0,1,0.125,0.353553
2,2,0,1,1,0,0,0,0,2,0.500,0.755929
3,3,0,0,0,0,2,0,0,0,0.250,0.707107
4,4,0,0,0,0,0,0,0,0,0.000,0.000000
5,5,0,0,0,0,0,0,0,0,0.000,0.000000
6,6,0,0,0,0,0,0,0,0,0.000,0.000000
7,7,0,0,0,0,0,0,0,0,0.000,0.000000
8,8,0,0,0,0,0,0,0,7,0.875,2.474874
9,9,0,0,9,0,1,60,0,0,8.750,20.940392


## 4. Stochastic Logistic Model

Here we follow the main reference *"A macroecological description of alternative stable states reproduces intra- and inter-host variability of gut microbiome", Silvia Zaoli and Jacopo Grilli*.

### Dissimilarity
    